# ServiceX Example

In [24]:
import requests
from minio import Minio
import tempfile
import pyarrow.parquet as pq



## Submit the Transform Request
We will create a REST request that specifies a DID along with a list of columns we want extracted.

We also tell ServiceX that we want the resulting columns to be stored as parquet files in the object store

In [2]:
    servicex_endpoint = "http://localhost:5000/servicex"
    response = requests.post(servicex_endpoint+"/transformation", json={
        "did": "mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00",
        "columns": "Electrons.pt(), Electrons.eta(), Electrons.phi(), Electrons.e(), Muons.pt(), Muons.eta(), Muons.phi(), Muons.e()",
        "image": "sslhep/servicex-transformer:latest",
        "result-destination": "object-store",
        "result-format": "parquet",
        "chunk-size": 7000,
        "workers": 1
    })

    print(response.json())
    request_id = response.json()["request_id"]
    status_endpoint = servicex_endpoint+"/transformation/{}/status".format(request_id)


{'request_id': '51722f8b-c6da-4821-8e4d-7325d0195aa2'}


## Wait for the Transform to Complete

In [8]:
status = requests.get(status_endpoint).json()
print("We have processed {} files there are {} remainng".format(status['files-processed'], status['files-remaining']))


We have processed 0 files there are 1 remainng


In [19]:
minio_endpoint = "localhost:9000"
minio_client = Minio(minio_endpoint,
                 access_key='miniouser',
                 secret_key='leftfoot1',
                 secure=False)
objects = minio_client.list_objects(request_id)
sample_file = list([file.object_name for file in objects])[0]
print(sample_file)

https:::recastwww.web.cern.ch:recastwww:data:reana-recast-demo:mc15_13TeV.123456.cap_recast_demo_signal_one.root


In [28]:
from IPython.display import display, HTML

with tempfile.TemporaryDirectory() as tmpdirname:
    minio_client.fget_object(request_id,
                             sample_file,
                             sample_file)
    pa_table = pq.read_table(sample_file)

    display(pa_table.to_pandas())
    

,Electrons_pt,Muons_eta,Muons_e,Electrons_eta,Muons_pt,Muons_phi,Electrons_phi,Electrons_e
0,"[664153.4375, 418500.84375, 325489.0, 304594.0...","[1.4914162158966064, 0.9966673851013184, -2.41...","[9264.535010560368, 10926.294491885757, 45149....","[-0.6133012175559998, -0.5223830938339233, 0.1...","[3968.750732421875, 7098.47802734375, 7998.909...","[-1.227807641029358, -0.10827206820249557, 0.8...","[-2.1278438568115234, 2.1374762058258057, 0.42...","[793024.8439972721, 476912.32395464927, 330326..."
1,"[570912.6875, 340445.5, 206510.3125, 130710.49...",[-0.3639054298400879],[42400.60110246442],"[-0.5060649514198303, 1.0624754428863525, 0.48...",[39739.9765625],[1.091120958328247],"[-2.0467095375061035, -2.7553279399871826, 0.1...","[645592.1441693598, 551372.6163316212, 230966...."
2,"[327393.46875, 240906.640625, 189487.109375, 1...","[0.07158683240413666, 0.3177592158317566, 0.01...","[4542.4492146670245, 196415.70221873414, 4649....","[-0.9719388484954834, -0.38464611768722534, -0...","[4529.60888671875, 186900.265625, 4647.8344726...","[-0.08602618426084518, 0.012212967500090599, 0...","[2.4008948802948, -1.525801420211792, 2.565304...","[494595.4021709691, 258948.84012466128, 189494..."
3,"[329687.40625, 264948.375, 153116.78125, 13404...","[0.3499639928340912, 0.46276557445526123, -0.0...","[175274.78021675578, 419621.14965282596, 3253....","[0.6903679370880127, -1.4229635000228882, 0.94...","[165063.125, 378377.75, 3247.887451171875, 406...","[2.304243326187134, -1.6472728252410889, 0.527...","[-1.6702631711959839, 0.26247426867485046, 2.7...","[411423.63735643553, 581614.3733124338, 227468..."
4,"[182541.015625, 146831.375, 140427.921875, 125...","[0.5504080057144165, 0.6646899580955505, 0.219...","[60735.28992450197, 589649.1346844176, 15442.5...","[0.5437236428260803, 0.6253201365470886, 1.372...","[52569.23046875, 479717.0, 15077.7158203125]","[1.1996105909347534, -1.4682575464248657, -0.8...","[1.6436755657196045, -1.5519003868103027, -2.9...","[210195.14996567575, 176486.4853893918, 294820..."
...,...,...,...,...,...,...,...,...
9795,"[582739.5625, 326050.5, 331992.71875, 134615.2...","[0.22276104986667633, 1.699975609779358, -1.72...","[110331.82780710579, 5930.459132210805, 6210.9...","[-0.852125346660614, 0.22676552832126617, -0.7...","[107649.7890625, 2096.53173828125, 2146.769775...","[-1.9117839336395264, 2.2606263160705566, 2.37...","[1.798552393913269, -1.9003254175186157, 1.774...","[807424.1806584733, 334469.668401183, 423399.1..."
9796,"[458738.25, 399355.09375, 196414.609375, 16263...",[-0.0013790572993457317],[5383.218384831052],"[-0.4057641923427582, 0.07747243344783783, 0.7...",[5382.17626953125],[-1.121288537979126],"[2.992771863937378, -0.7303695678710938, 1.015...","[497023.62696927355, 400554.1535302422, 248590..."
9797,"[408538.34375, 229417.109375, 161219.609375, 1...","[0.7550489902496338, -0.37565305829048157, -1....","[58921.05492651263, 5150.149489555235, 5758.26...","[0.4747697412967682, 0.08857348561286926, 0.23...","[45363.88671875, 4805.9619140625, 2284.4978027...","[-0.49022841453552246, 1.8972296714782715, 2.7...","[-2.1312596797943115, 1.2088642120361328, -2.1...","[455453.30370199465, 230317.6165767007, 165543..."
9798,"[801421.875, 375173.3125, 286641.15625, 138267...",[-0.13840670883655548],[3854.3190712734227],"[-0.4224384129047394, -0.48981884121894836, -0...",[3816.25927734375],[1.2501146793365479],"[-0.4565953016281128, -2.8471243381500244, -3....","[874000.1926816264, 421086.6332812778, 321683...."
